In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1 --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-api-core 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-aiplatform 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [2]:
import os
import textwrap
import pathlib

from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse
from groq import Groq

GROQ_API_KEY = "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"
client = Groq(
    api_key = GROQ_API_KEY,
)

def print_response(response):
    response_txt = response['result']
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [3]:
!mkdir data
!gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

Downloading...
From: https://drive.google.com/uc?id=1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9
To: /content/data/meta-earnings.pdf
100% 160k/160k [00:00<00:00, 61.4MB/s]


## Document Parsing

In [4]:
instruction = """The provided document is Meta First Quarter 2024 Results.
This form provides detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key='llx-LhWfJSzW2UgUy3Pa1JtPh3Vt1A5AAD5YeFfjGBRV6vkU7Lrw',
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("./data/meta-earnings.pdf")


Started parsing the file under job_id cac11eca-9df1-469e-a3de-a11c283df430


In [5]:
parsed_doc = llama_parse_documents[0]

In [6]:
Markdown(parsed_doc.text[:4096])

# Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

# First Quarter 2024 Financial Highlights

|In millions, except percentages and per share amounts|Three Months Ended March 31,| |2024| |2023|% Change|
|---|---|---|---|---|---|---|
|Revenue|$36,455|$28,645|27%| | | |
|Costs and expenses|$22,637|$21,418|6%| | | |
|Income from operations|$13,818|$7,227|91%| | | |
|Operating margin|38%|25%| | | | |
|Provision for income taxes|$1,814|$1,598|14%| | | |
|Effective tax rate|13%|22%| | | | |
|Net income|$12,369|$5,709|117%| | | |
|Diluted earnings per share (EPS)|$4.71|$2.20|114%| | | |

# First Quarter 2024 Operational and Other Financial Highlights

- Family daily active people (DAP) – DAP was 3.24 billion on average for March 2024, an increase of 7% year-over-year.
- Ad impressions – Ad impressions delivered across our Family of Apps increased by 20% year-over-year.
- Average price per ad – Average price per ad increased by 6% year-over-year.
- Revenue – Total revenue and revenue on a constant currency basis were $36.46 billion and $36.35 billion, respectively, both of which increased by 27% year-over-year.
- Costs and expenses – Total costs and expenses were $22.64 billion, an increase of 6% year-over-year.
- Capital expenditures – Capital expenditures, including principal payments on finance leases, were $6.72 billion.
- Capital return program – Share repurchases were $14.64 billion of our Class A common stock and dividends payments were $1.27 billion.
- Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $58.12 billion as of March 31, 2024. Free cash flow was $12.53 billion.
- Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.
---
CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in pe range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growp, based on current exchange rates.
We expect full-year 2024 total expenses to be in pe range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to furper scale our ecosystem.
We anticipate our full-year 2024 capital expenditures will be in pe range of $35-40 billion, increased from our prior range of $30-37 billion as we continue to accelerate our infrastructure investments to support our artificial intelligence (AI) roadmap. While we are not providing guidance for years beyond 2024, we expect capital expenditures will continue to increase next year as we invest aggressively to support our ambitious AI research and product development efforts.
Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in pe mid-teens.
In addition, we continue to monitor an active regulatory landscape, including pe increasing legal and regulatory headwinds in pe EU and pe U.S. pat could significantly impact our business and our financial results.
Q1 was a good start to pe year. We're seeing strong momentum wipin our Family of Apps and are making important progress on our longer-term AI and Reality Labs initiatives pat have pe potential to transform pe way people interact wip our services over pe coming years.
---
Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnin

## Vector Embeddings

In [7]:
from pathlib import Path

document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

In [8]:
loader = UnstructuredMarkdownLoader(document_path)

In [9]:
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

11

In [11]:
print(docs[0].page_content)


Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts Three Months Ended March 31, 2024 2023 % Change Revenue $36,455 $28,645 27% Costs and expenses $22,637 $21,418 6% Income from operations $13,818 $7,227 91% Operating margin 38% 25% Provision for income taxes $1,814 $1,598 14% Effective tax rate 13% 22% Net income $12,369 $5,709 117% Diluted earnings per share (EPS) $4.71 $2.20 114%

First Quarter 2024 Operational and Other Financial Highlights

Family daily active peo

In [12]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [13]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [14]:
%%time
query = "What is the most important innovation from Meta?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 513 ms, sys: 1.76 ms, total: 515 ms
Wall time: 715 ms


In [15]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

score: 0.6154119568600498
--------------------------------------------------------------------------------

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

score: 0.5709373825059564
--------------------------------------------------------------------------------

text: Reconciliation of cash, cash equivalents, and restricted cash to the condensed consolidated balance sheets

Cash and cash equivalents $32,307 $11,551 Restricted cash, included in prepaid expenses and other current assets 84 224 Restricted cash, inclu

In [16]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 513 ms, sys: 94.7 ms, total: 608 ms
Wall time: 663 ms


In [17]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: 613a2a651f6a4b9aadfa630063414b54

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

--------------------------------------------------------------------------------

id: 9da8bd8f5ca743d289c25089cb08ca1a

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

--------------------------------------------------------------------------------

id: fe003397fc744b538b1f879a3cac1577

text: Reconciliation of cash, cash equivalents, and restricted cash to the condensed consolidated balance sheets

Cash and cash equivalents $32,307 $11,551 Restricted cash, included in prepaid e

## Reranking

In [19]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 65.5MiB/s]


In [20]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 3 s, sys: 186 ms, total: 3.18 s
Wall time: 3.32 s


3

In [21]:

for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()


id: 613a2a651f6a4b9aadfa630063414b54

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

score: 0.14779266715049744
--------------------------------------------------------------------------------

id: 9da8bd8f5ca743d289c25089cb08ca1a

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

score: 0.0076220473274588585
--------------------------------------------------------------------------------

id: 97d8d48799394da5b85e945565d1b1c5

text: META PLATFORMS, INC.

CONDENSED CONSOLIDATED BALANCE SHEETS

(In millions)

(Unaudited)

March 31, 2024 December 31, 2023 Assets Cur

## Q&A Over Document

In [26]:
llm = ChatGroq(groq_api_key=GROQ_API_KEY, temperature=0, model_name="llama3-70b-8192")

In [27]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [28]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [29]:
%%time
response = qa.invoke("What is the most significant innovation from Meta?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts Three Months Ended March 31, 2024 2023 % Change Revenue $36,455 $28,645 27% Costs and expenses $22,637 $21,418 6% 

In [30]:
print_response(response)

Based on the provided information, the most significant innovation from Meta is the new version of
Meta AI with Llama 3, which is mentioned in the quote from Mark Zuckerberg, Meta founder and CEO.
This suggests that Meta is making progress in building the world's leading AI.

Additionally, the information highlights Meta's focus on building the metaverse, which is mentioned
as one of the areas where the company is making steady progress. However, it does not provide
further details on what specific innovations or developments are being made in this area.


In [31]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [35]:
%%time
response = qa.invoke("What is the revenue for 2024 and % change?")

Running pairwise ranking..
CPU times: user 3.01 s, sys: 5.28 ms, total: 3.02 s
Wall time: 3.95 s


In [36]:
Markdown(response["result"])

**Revenue for 2024 and % Change:**

The revenue for the first quarter of 2024 is $36.46 billion, which represents a 27% year-over-year increase.

**Additional Helpful Information:**

* The revenue on a constant currency basis is $36.35 billion, also a 27% year-over-year increase.
* The company expects second-quarter 2024 total revenue to be in the range of $36.5-39 billion.

In [37]:
%%time
response = qa.invoke("What is the revenue for 2023?")

Running pairwise ranking..
CPU times: user 2.94 s, sys: 6.45 ms, total: 2.94 s
Wall time: 3.84 s


In [38]:
print_response(response)

The revenue for 2023 is not explicitly stated in the provided information. However, we can infer it
from the "First Quarter 2024 Financial Highlights" section, which mentions that the revenue for Q1
2024 is $36,455 million, with a 27% year-over-year increase. This implies that the revenue for Q1
2023 was $28,645 million.

Here's the relevant information:

* Revenue (Q1 2024): $36,455 million
* Year-over-year increase: 27%
* Revenue (Q1 2023): $28,645 million


In [39]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 2.98 s, sys: 10.9 ms, total: 2.99 s
Wall time: 4.02 s


In [40]:
print_response(response)

Based on the provided information, we can calculate the revenue minus the costs and expenses for
2024 as follows:

**Revenue:**
Total revenue for Q1 2024 is $36,455 million.

**Costs and Expenses:**
We don't have the exact costs and expenses for Q1 2024, but we can calculate the income from
operations, which is the revenue minus the costs and expenses.

**Income from Operations:**
Total income from operations for Q1 2024 is $13,818 million.

**Revenue minus Costs and Expenses:**
Therefore, the revenue minus the costs and expenses for Q1 2024 is $13,818 million.

**Additional Information:**

* The company expects full-year 2024 total expenses to be in the range of $96-99 billion.
* The company expects full-year 2024 capital expenditures to be in the range of $35-40 billion.

Please note that these calculations are based on the provided information and might not reflect the
company's actual financial performance for the entire year 2024.


In [41]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 3.19 s, sys: 83.8 ms, total: 3.27 s
Wall time: 4.75 s


In [42]:
print_response(response)

To calculate the revenue minus the costs and expenses for 2023, we need to find the total revenue
and total income from operations for 2023.

From the Segment Information section, we can see that the total revenue for 2023 is $28,645.

From the same section, we can see that the total income from operations for 2023 is $7,227.

However, the question asks for revenue minus costs and expenses, which is not exactly the same as
income from operations. Income from operations is revenue minus operating expenses, but it does not
include non-operating income and expenses.

To calculate revenue minus costs and expenses, we can use the net income figure, which is $5,709 for
2023. Net income is revenue minus all costs and expenses, including operating and non-operating
expenses.

Therefore, the revenue minus costs and expenses for 2023 is $5,709.

Additional helpful information:

* The net income margin for 2023 is 20% ($5,709 ÷ $28,645).
* The operating income margin for 2023 is 25% ($7,227 ÷ $28

In [43]:
%%time
response = qa.invoke("What is the expected revenue for the second quarter of 2024?")

Running pairwise ranking..
CPU times: user 3.21 s, sys: 3.87 ms, total: 3.21 s
Wall time: 4.39 s


In [44]:
Markdown(response["result"])

**Answer:** The expected revenue for the second quarter of 2024 is in the range of $36.5-39 billion.

**Additional helpful information:**

* The company's guidance assumes a 1% headwind to year-over-year total revenue growth due to foreign currency exchange rates.
* The company has reported a strong start to the year, with revenue increasing by 27% year-over-year in the first quarter of 2024.

In [45]:
%%time
response = qa.invoke("What is the overall outlook of Q1 2024?")

Running pairwise ranking..
CPU times: user 3.13 s, sys: 5.55 ms, total: 3.13 s
Wall time: 4.17 s


In [46]:
print_response(response)

**Overall Outlook of Q1 2024:**

The overall outlook of Q1 2024 is positive. According to Mark Zuckerberg, Meta's founder and CEO,
"It's been a good start to the year." The company reported strong financial results, with revenue
increasing by 27% year-over-year to $36.46 billion, and net income increasing by 117% year-over-year
to $12.37 billion.

**Additional Helpful Information:**

* Family daily active people (DAP) increased by 7% year-over-year to 3.24 billion on average for
March 2024.
* Ad impressions delivered across the Family of Apps increased by 20% year-over-year.
* Average price per ad increased by 6% year-over-year.
* The company saw strong momentum in its Family of Apps and made important progress on its longer-
term AI and Reality Labs initiatives.
